In [ ]:
import numpy as np
import pennylane as qml
from pennylane import X, Y, Z


In [3]:
np.abs(np.sin(1j))**2+np.abs(np.cos(1j))**2

3.7621956910836314

In [5]:
np.log(2)

0.6931471805599453

In [ ]:

su2 = [1j * Y(0) @ X(1)]

coeffs = [1.]                           # some real coefficients
exponent = qml.dot(coeffs, su2)                 # linear combination of operators
U = qml.math.expm(exponent.matrix())            # compute matrix exponent of lin. comb.
print(np.allclose(U.conj().T @ U, np.eye(2)))   # check that result is unitary UU* = 1

In [18]:
U

array([[0.54030231+0.84147098j, 0.        +0.j        ],
       [0.        +0.j        , 0.54030231-0.84147098j]])

In [2]:
coeffs = [1., 2.+ 1j, 3.]                       # some complex coefficients
exponent = qml.dot(coeffs, su2)
U = qml.math.expm(exponent.matrix())
print(np.allclose(U.conj().T @ U, np.eye(2)))   # result is not unitary anymore

False


In [3]:
print(qml.commutator(1j * X(0), 1j * Y(0)))

-2j * Z(0)


In [4]:
list_ops = [1j * X(0), 1j * Y(0), 1j * Z(0)]
for op1 in list_ops:
    for op2 in list_ops:
        print(qml.commutator(op1, op2))

0 * I()
-2j * Z(0)
2j * Y(0)
2j * Z(0)
0 * I()
-2j * X(0)
-2j * Y(0)
2j * X(0)
0 * I()


In [11]:
dla = qml.lie_closure([ X(0), Y(0)])
dla

[X(0), Y(0), Z(0)]

In [12]:
dla = qml.lie_closure([Z(0),Y(0)])
dla

[Z(0), Y(0), -1.0 * X(0)]

In [13]:
dla = qml.lie_closure([X(0),Z(0)])
dla

[X(0), Z(0), -1.0 * Y(0)]

In [22]:
import numpy as np

# Define Pauli matrices
X = np.array([[0, 1], [1, 0]], dtype=complex)
Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
Z = np.array([[1, 0], [0, -1]], dtype=complex)
I = np.eye(2, dtype=complex)

# Tensor product helper
def kron(a, b):
    return np.kron(a, b)

# Define two-qubit Ising generators: Z0, Z1, X0X1
Z0 = kron(Z, I)
Z1 = kron(I, Z)
X0X1 = kron(X, X)

# Convert to anti-Hermitian generators: -i * H
gens = [-1j * Z0, -1j * Z1, -1j * X0X1]

# Commutator
def comm(A, B):
    return A @ B - B @ A

# Initialize basis
basis = gens.copy()
tol = 1e-8

def is_independent(M, basis):
    # Check if M is linearly independent of basis
    mats = basis + [M]
    # Flatten and stack
    stacked = np.column_stack([m.flatten() for m in mats])
    # Compute singular values
    s = np.linalg.svd(stacked, compute_uv=False)
    # If smallest singular value > tol, new direction added
    return s[-1] > tol

# Grow Lie algebra
changed = True
while changed:
    changed = False
    for A in basis.copy():
        for B in basis.copy():
            C = comm(A, B)
            if np.linalg.norm(C) < tol:
                continue
            if is_independent(C, basis):
                basis.append(C)
                changed = True

# Output dimension
print("Generated DLA dimension:", len(basis))
basis

Generated DLA dimension: 6


[array([[0.-1.j, 0.-0.j, 0.-0.j, 0.-0.j],
        [0.-0.j, 0.-1.j, 0.-0.j, 0.-0.j],
        [0.-0.j, 0.-0.j, 0.+1.j, 0.+0.j],
        [0.-0.j, 0.-0.j, 0.+0.j, 0.+1.j]]),
 array([[0.-1.j, 0.-0.j, 0.-0.j, 0.-0.j],
        [0.-0.j, 0.+1.j, 0.-0.j, 0.+0.j],
        [0.-0.j, 0.-0.j, 0.-1.j, 0.-0.j],
        [0.-0.j, 0.+0.j, 0.-0.j, 0.+1.j]]),
 array([[0.-0.j, 0.-0.j, 0.-0.j, 0.-1.j],
        [0.-0.j, 0.-0.j, 0.-1.j, 0.-0.j],
        [0.-0.j, 0.-1.j, 0.-0.j, 0.-0.j],
        [0.-1.j, 0.-0.j, 0.-0.j, 0.-0.j]]),
 array([[ 0.+0.j,  0.+0.j,  0.+0.j, -2.+0.j],
        [ 0.+0.j,  0.+0.j, -2.+0.j,  0.+0.j],
        [ 0.+0.j,  2.+0.j,  0.+0.j,  0.+0.j],
        [ 2.+0.j,  0.+0.j,  0.+0.j,  0.+0.j]]),
 array([[ 0.+0.j,  0.+0.j,  0.+0.j, -2.+0.j],
        [ 0.+0.j,  0.+0.j,  2.+0.j,  0.+0.j],
        [ 0.+0.j, -2.+0.j,  0.+0.j,  0.+0.j],
        [ 2.+0.j,  0.+0.j,  0.+0.j,  0.+0.j]]),
 array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+4.j],
        [0.+0.j, 0.+0.j, 0.-4.j, 0.+0.j],
        [0.+0.j, 0.-4.j, 0.+0.j, 0